In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack

In [ ]:

# Load cleaned datasets
trainingSet = pd.read_csv('/content/train_fixed_cleaned.csv')
testingSet = pd.read_csv('/content/test.csv')

ParserError: Error tokenizing data. C error: EOF inside string starting at row 93678

In [ ]:
file_path = '/content/train_fixed_cleaned.csv'

# Check for rows with an odd number of quotes
with open(file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if line.count('"') % 2 != 0:  # Check for odd number of quotes
            print(f"Issue with quotes on line {i + 1}: {line}")
            break  # Stop at first issue to investigate


Issue with quotes on line 269827: 1044134,B000M8NMVE,A34MWK16CNJAVN,15,16,1190937600,Danger Is His Business,"I remember watching this show before school. Only in B&W;, (okay, so I'm old) and never missed an episode. Now, finally, I can re-live those early mornings in color, and in pristine clarity. I have to say, as soon as the menu came up with the theme song, my old grey cells kicked in, and I had a smile from ear to ear.Like ""The Archies"", ""Groovie Goolies"" and ""Top Cat"", the REAL openings are there, captured in their FULL format. Unlike ""Magilla Gorila""


In [ ]:
import csv

input_path = '/content/train_fixed_cleaned.csv'
output_path = '/content/train_fixed_cleaned_corrected.csv'

# Determine the expected number of columns based on the header row
with open(input_path, 'r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    header = next(reader)
    expected_columns = len(header)

# Process the file, fixing problematic rows
with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    writer.writerow(header)  # Write the header to the output file

    for i, row in enumerate(reader, start=2):  # Start at line 2 to account for header
        # If row has fewer or more columns than expected, attempt to fix it
        if len(row) != expected_columns:
            fixed_row = []

            for field in row:
                # Correct fields with unbalanced quotes
                if field.count('"') % 2 != 0:
                    field = field.replace('"', '')  # Remove stray quotes
                fixed_row.append(field)

            # Flatten the row to the expected length
            fixed_row = fixed_row[:expected_columns] + [''] * (expected_columns - len(fixed_row))
            writer.writerow(fixed_row)
        else:
            writer.writerow(row)


In [ ]:
import pandas as pd

# Load cleaned file
trainingSet = pd.read_csv('/content/train_fixed_cleaned_corrected.csv')
print("File loaded successfully with corrected rows.")


File loaded successfully with corrected rows.


<ipython-input-6-9e64ff4d9e12>:4: DtypeWarning: Columns (0,3,4,5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  trainingSet = pd.read_csv('/content/train_fixed_cleaned_corrected.csv')


In [ ]:
testingSet = pd.read_csv('/content/test.csv')

In [ ]:
# Ensure that HelpfulnessNumerator and HelpfulnessDenominator are numeric
trainingSet['HelpfulnessNumerator'] = pd.to_numeric(trainingSet['HelpfulnessNumerator'], errors='coerce').fillna(0)
trainingSet['HelpfulnessDenominator'] = pd.to_numeric(trainingSet['HelpfulnessDenominator'], errors='coerce').fillna(1)

# Drop rows where 'Score' is missing
trainingSet.dropna(subset=['Score'], inplace=True)

# Feature engineering on training data
trainingSet['HelpfulnessRatio'] = trainingSet['HelpfulnessNumerator'] / trainingSet['HelpfulnessDenominator']
trainingSet['ReviewLength'] = trainingSet['Text'].apply(lambda x: len(str(x)))


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Create the Document-Term Matrix
count_vect = CountVectorizer(max_features=3000)
count_matrix = count_vect.fit_transform(trainingSet['Text'].fillna(""))

# Step 2: Apply Truncated SVD for dimensionality reduction
svd = TruncatedSVD(n_components=100, random_state=42)  # Adjust components based on memory constraints
text_features = svd.fit_transform(count_matrix)


In [ ]:

# Clustering on reduced features
kmeans = KMeans(n_clusters=5, random_state=42)
trainingSet['ClusterLabel'] = kmeans.fit_predict(text_features)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import csr_matrix

# Ensure the 'Score' column is an integer type
trainingSet['Score'] = pd.to_numeric(trainingSet['Score'], errors='coerce').fillna(0).astype(int)

# Prepare numerical features and shift to non-negative values
X_numerical = trainingSet[['HelpfulnessRatio', 'ReviewLength', 'ClusterLabel']].fillna(0)

# Apply MinMaxScaler to ensure non-negative values for numerical features
scaler_numerical = MinMaxScaler()
X_numerical_scaled = scaler_numerical.fit_transform(X_numerical)

# Apply MinMaxScaler to ensure non-negative values for text features
scaler_text = MinMaxScaler()
text_features_scaled = scaler_text.fit_transform(text_features)

# Combine scaled numerical and text features
X = hstack([csr_matrix(X_numerical_scaled), csr_matrix(text_features_scaled)])
y = trainingSet['Score']

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training with Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_val)

# Validation Results
print("Naive Bayes Accuracy:", accuracy_score(y_val, y_pred_nb))
print("Naive Bayes Classification Report:\n", classification_report(y_val, y_pred_nb))


Naive Bayes Accuracy: 0.5328886451163916


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Naive Bayes Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00     10474
           2       0.00      0.00      0.00     10392
           3       0.00      0.00      0.00     20076
           4       0.00      0.00      0.00     38260
           5       0.53      1.00      0.70     90355

    accuracy                           0.53    169557
   macro avg       0.11      0.20      0.14    169557
weighted avg       0.28      0.53      0.37    169557



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Validation Results
print("Naive Bayes Accuracy:", accuracy_score(y_val, y_pred_nb))
print("Naive Bayes Classification Report:\n", classification_report(y_val, y_pred_nb))


Naive Bayes Accuracy: 0.5328886451163916
Naive Bayes Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00     10474
           2       0.00      0.00      0.00     10392
           3       0.00      0.00      0.00     20076
           4       0.00      0.00      0.00     38260
           5       0.53      1.00      0.70     90355

    accuracy                           0.53    169557
   macro avg       0.11      0.20      0.14    169557
weighted avg       0.28      0.53      0.37    169557



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Merge test set with necessary columns from training data
test_data = testingSet.merge(trainingSet[['Id', 'HelpfulnessRatio', 'ReviewLength', 'ClusterLabel', 'Text']], on='Id', how='left')

# Fill or recompute missing features in test set
test_data['HelpfulnessRatio'] = test_data['HelpfulnessRatio'].fillna(0)
test_data['ReviewLength'] = test_data['Text'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
test_data['Text'] = test_data['Text'].fillna("")

# Transform text features in test data
test_count_matrix = count_vect.transform(test_data['Text'])
test_text_features = svd.transform(test_count_matrix)

# Prepare test features
X_test_numerical = csr_matrix(test_data[['HelpfulnessRatio', 'ReviewLength', 'ClusterLabel']].fillna(0).values)
X_test = hstack([X_test_numerical, csr_matrix(test_text_features)])

# Predict on the test set
test_predictions = nb_model.predict(X_test)




In [ ]:
# Prepare the final submission
testingSet['Score'] = test_predictions
submission = testingSet[['Id', 'Score']]
submission.to_csv("/content/submission_7.csv", index=False)
print("Submission file saved.")

Submission file saved.
